In [35]:
import pandas as pd
from pathlib import Path
import yaml
from datetime import timedelta

def yaml_to_csv(input_folder, output_file):
    
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    combined_data = []

    for month_path in Path(input_folder).iterdir():
        if not month_path.is_dir():
            continue

        for file_path in month_path.glob('*.yaml'):
            with open(file_path, 'r') as file:
                data = yaml.safe_load(file)
                if isinstance(data, list):
                    combined_data.append(pd.DataFrame(data))

    if combined_data:
        combined_df = pd.concat(combined_data, ignore_index=True)
        combined_df.to_csv(output_file, index=False)
        print(f"Data saved to {output_file}")
    else:
        print("No data found.")

def analyze_and_merge_data(input_csv):
    """Analyze data for yearly returns, volatility, market summary, and merge results."""
    try:
        
        print("Loading data...")
        df = pd.read_csv(input_csv)
        
        required_columns = {'date', 'open', 'close', 'Ticker'}
        if not required_columns.issubset(df.columns):
            raise ValueError(f"Input CSV must contain columns: {required_columns}")
        
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df['date'].dt.year

        # Yearly analysis
        print("Performing yearly analysis...")
        yearly_data = df.groupby(['Ticker', 'year']).agg(
            first_open=('open', 'first'),
            last_close=('close', 'last')
        ).reset_index()

        yearly_data['yearly_return'] = ((yearly_data['last_close'] - yearly_data['first_open']) / 
                                        yearly_data['first_open']) * 100

        # Identify top 10 green and red stocks
        top_10_green = yearly_data.nlargest(10, 'yearly_return').assign(rank='Top Green')
        top_10_red = yearly_data.nsmallest(10, 'yearly_return').assign(rank='Top Red')

        # Volatility analysis
        print("Calculating volatility...")
        df['daily_return'] = df.groupby('Ticker')['close'].pct_change()
        volatility = df.groupby('Ticker')['daily_return'].std().reset_index().rename(
            columns={'daily_return': 'volatility'}
        )

        # Calculate cumulative return
        df['cumulative_return'] = df.groupby('Ticker')['daily_return'].cumsum()

        # Monthly returns
        print("Calculating monthly returns...")
        df['month'] = df['date'].dt.to_period('M')
        monthly_data = df.groupby(['Ticker', 'month']).agg(
            open=('open', 'first'),
            close=('close', 'last')
        ).reset_index()
        monthly_data['monthly_return'] = ((monthly_data['close'] - monthly_data['open']) / 
                                          monthly_data['open']) * 100
        
        print("Merging data into a single DataFrame...")
        merged_data = pd.merge(df, yearly_data, on=['Ticker', 'year'], how='left')
        merged_data = pd.merge(merged_data, volatility, on='Ticker', how='left')
        merged_data = pd.merge(merged_data, monthly_data, on=['Ticker', 'month'], how='left')
                     
        merged_data.to_csv('merged_analysis.csv', index=False)
        print("Merged data saved as 'merged_analysis.csv'.")

        return merged_data

    except Exception as e:
        print(f"Error during analysis: {e}")
        return None

if __name__ == "__main__":
    input_folder = "D:/projects/stock/data"
    output_file = "D:/projects/stock/output/combined.csv"
    
    yaml_to_csv(input_folder, output_file)
    
    merged_data = analyze_and_merge_data(output_file)


Data saved to D:/projects/stock/output/combined.csv
Loading data...
Performing yearly analysis...
Calculating volatility...
Calculating monthly returns...
Merging data into a single DataFrame...
Merged data saved as 'merged_analysis.csv'.


In [49]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14200 entries, 0 to 14199
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Ticker             14200 non-null  object        
 1   close_x            14200 non-null  float64       
 2   date               14200 non-null  datetime64[ns]
 3   high               14200 non-null  float64       
 4   low                14200 non-null  float64       
 5   month              14200 non-null  period[M]     
 6   open_x             14200 non-null  float64       
 7   volume             14200 non-null  int64         
 8   year               14200 non-null  int32         
 9   daily_return       14150 non-null  float64       
 10  cumulative_return  14150 non-null  float64       
 11  first_open         14200 non-null  float64       
 12  last_close         14200 non-null  float64       
 13  yearly_return      14200 non-null  float64       
 14  volati

In [47]:
missing_data_summary = merged_data.isnull().sum() 
print(missing_data_summary)

Missing data summary:
Ticker                0
close_x               0
date                  0
high                  0
low                   0
month                 0
open_x                0
volume                0
year                  0
daily_return         50
cumulative_return    50
first_open            0
last_close            0
yearly_return         0
volatility            0
open_y                0
close_y               0
monthly_return        0
dtype: int64


In [55]:
merged_data.fillna({'daily_return': 0, 'cumulative_return': 0}, inplace=True)

print(merged_data[['daily_return', 'cumulative_return']].isnull().sum())

daily_return         0
cumulative_return    0
dtype: int64


In [59]:
merged_data.tail()

,Ticker,close_x,date,high,low,month,open_x,volume,year,daily_return,cumulative_return,first_open,last_close,yearly_return,volatility,open_y,close_y,monthly_return
14195,HINDALCO,652.10,2024-11-22 05:30:00,657.3,647.40,2024-11,652.95,3586402,2024,0.006250,0.361646,614.50,652.10,6.118796,0.019587,687.50,652.10,-5.149091
14196,HEROMOTOCO,4794.10,2024-11-22 05:30:00,4808.4,4742.00,2024-11,4778.40,394712,2024,0.005506,0.502506,4139.55,4794.10,15.812105,0.016563,5039.70,4794.10,-4.873306
14197,AXISBANK,1142.40,2024-11-22 05:30:00,1147.9,1127.55,2024-11,1136.65,16687505,2024,0.002853,0.127365,1095.00,1142.40,4.328767,0.015625,1164.05,1142.40,-1.859886
14198,HDFCBANK,1745.60,2024-11-22 05:30:00,1754.3,1729.55,2024-11,1743.55,12386806,2024,0.002527,0.172242,1706.00,1745.60,2.321219,0.013490,1733.00,1745.60,0.727063
14199,BAJAJ-AUTO,9481.65,2024-11-22 05:30:00,9602.9,9444.10,2024-11,9545.00,619823,2024,-0.002457,0.681613,6770.00,9481.65,40.053914,0.017609,9905.00,9481.65,-4.274104


In [85]:
merged_data.to_csv('df.csv', index=False)  
df1 = merged_data  

In [87]:
df1 = df1.rename(columns={
    'close_x': 'close',
    'open_x': 'open',
})
print(df1.head())  

       Ticker    close                date    high      low    month    open  \
0        SBIN   602.95 2023-10-03 05:30:00   604.9   589.60  2023-10   596.6   
1  BAJFINANCE  7967.60 2023-10-03 05:30:00  7975.5  7755.00  2023-10  7780.8   
2       TITAN  3196.25 2023-10-03 05:30:00  3212.5  3114.40  2023-10  3148.8   
3         ITC   439.75 2023-10-03 05:30:00   442.9   439.25  2023-10   441.0   
4         TCS  3513.85 2023-10-03 05:30:00  3534.2  3480.10  2023-10  3534.2   

     volume  year  daily_return  cumulative_return  first_open  last_close  \
0  15322196  2023           0.0                0.0       596.6      642.05   
1    944555  2023           0.0                0.0      7780.8     7327.75   
2   1007308  2023           0.0                0.0      3148.8     3675.45   
3   7481883  2023           0.0                0.0       441.0      462.10   
4   1948148  2023           0.0                0.0      3534.2     3793.40   

   yearly_return  volatility  open_y  close_y  mon

In [93]:
df1.to_csv("df1.csv",index=False)


In [97]:
df1.head()

,Ticker,close,date,high,low,month,open,volume,year,daily_return,cumulative_return,first_open,last_close,yearly_return,volatility,open_y,close_y,monthly_return
0,SBIN,602.95,2023-10-03 05:30:00,604.9,589.60,2023-10,596.6,15322196,2023,0.0,0.0,596.6,642.05,7.618170,0.017939,596.6,565.55,-5.204492
1,BAJFINANCE,7967.60,2023-10-03 05:30:00,7975.5,7755.00,2023-10,7780.8,944555,2023,0.0,0.0,7780.8,7327.75,-5.822666,0.015918,7780.8,7492.65,-3.703347
2,TITAN,3196.25,2023-10-03 05:30:00,3212.5,3114.40,2023-10,3148.8,1007308,2023,0.0,0.0,3148.8,3675.45,16.725419,0.014136,3148.8,3189.65,1.297320
3,ITC,439.75,2023-10-03 05:30:00,442.9,439.25,2023-10,441.0,7481883,2023,0.0,0.0,441.0,462.10,4.784580,0.011944,441.0,428.40,-2.857143
4,TCS,3513.85,2023-10-03 05:30:00,3534.2,3480.10,2023-10,3534.2,1948148,2023,0.0,0.0,3534.2,3793.40,7.334050,0.013162,3534.2,3368.75,-4.681399


In [99]:
df1= df1.drop(columns=['open_y',
    'close_y']   
)

In [101]:
df1.head()

,Ticker,close,date,high,low,month,open,volume,year,daily_return,cumulative_return,first_open,last_close,yearly_return,volatility,monthly_return
0,SBIN,602.95,2023-10-03 05:30:00,604.9,589.60,2023-10,596.6,15322196,2023,0.0,0.0,596.6,642.05,7.618170,0.017939,-5.204492
1,BAJFINANCE,7967.60,2023-10-03 05:30:00,7975.5,7755.00,2023-10,7780.8,944555,2023,0.0,0.0,7780.8,7327.75,-5.822666,0.015918,-3.703347
2,TITAN,3196.25,2023-10-03 05:30:00,3212.5,3114.40,2023-10,3148.8,1007308,2023,0.0,0.0,3148.8,3675.45,16.725419,0.014136,1.297320
3,ITC,439.75,2023-10-03 05:30:00,442.9,439.25,2023-10,441.0,7481883,2023,0.0,0.0,441.0,462.10,4.784580,0.011944,-2.857143
4,TCS,3513.85,2023-10-03 05:30:00,3534.2,3480.10,2023-10,3534.2,1948148,2023,0.0,0.0,3534.2,3793.40,7.334050,0.013162,-4.681399


In [103]:
df1.to_csv("df1.csv",index=False)

In [2]:
import pandas as pd

sector_data = pd.read_csv('D:/projects/stock/output/sector_data.csv')
combined_data = pd.read_csv('df1.csv')

sector_data['Symbol'] = sector_data['Symbol'].str.split(': ').str[-1]

merged_data = combined_data.merge(sector_data, left_on='Ticker', right_on='Symbol', how='left')

merged_data['date'] = pd.to_datetime(merged_data['date'])

merged_data['year'] = merged_data['date'].dt.year

df2= merged_data.to_csv('df2.csv',index=False)


In [6]:
merged_data.isnull().sum()

Ticker                  0
close                   0
date                    0
high                    0
low                     0
month                   0
open                    0
volume                  0
year                    0
daily_return            0
cumulative_return       0
first_open              0
last_close              0
yearly_return           0
volatility              0
monthly_return          0
COMPANY              1136
sector               1136
Symbol               1136
dtype: int64

In [26]:
import pandas as pd
from rapidfuzz import process

sector_data = pd.read_csv('D:/projects/stock/output/sector_data.csv')
combined_data = pd.read_csv('df1.csv')

sector_data['Symbol'] = sector_data['Symbol'].str.split(': ').str[-1]

def preprocess(text):
    if pd.isna(text):
        return ""
    return ''.join(e for e in text.lower() if e.isalnum())

sector_data['Processed_Symbol'] = sector_data['Symbol'].apply(preprocess)
combined_data['Processed_Ticker'] = combined_data['Ticker'].apply(preprocess)

symbols_list = sector_data['Processed_Symbol'].tolist()

#fuzzy matching
def fuzzy_match(ticker, symbols, threshold=70):
    match_result = process.extractOne(ticker, symbols)
    if match_result:
        match, score = match_result[:2]  # extractOne returns
        return match if score >= threshold else None
    return None

# to find the best matches
combined_data['Matched_Processed_Symbol'] = combined_data['Processed_Ticker'].apply(
    lambda x: fuzzy_match(x, symbols_list, threshold=70)
)

# Map matched processed symbols back to original symbols
symbol_mapping = sector_data.set_index('Processed_Symbol')['Symbol'].to_dict()
combined_data['Matched_Symbol'] = combined_data['Matched_Processed_Symbol'].map(symbol_mapping)

# Merge based on fuzzy-matched symbols and exact matches
merged_data = combined_data.merge(sector_data, left_on='Matched_Symbol', right_on='Symbol', how='left')

merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data['year'] = merged_data['date'].dt.year

merged_data.to_csv('df2.csv', index=False)


In [28]:
merged_data.isnull().sum()

Ticker                      0
close                       0
date                        0
high                        0
low                         0
month                       0
open                        0
volume                      0
year                        0
daily_return                0
cumulative_return           0
first_open                  0
last_close                  0
yearly_return               0
volatility                  0
monthly_return              0
Processed_Ticker            0
Matched_Processed_Symbol    0
Matched_Symbol              0
COMPANY                     0
sector                      0
Symbol                      0
Processed_Symbol            0
dtype: int64

In [42]:
merged_data=merged_data.drop(columns=['Matched_Processed_Symbol'])
merged_data.to_csv('df2.csv', index=False)

In [1]:
import pandas as pd
import pymysql

def store_csv_in_mysql(csv_file):    
    df = pd.read_csv(csv_file)
    
    df.rename(columns={
        'COMPANY': 'company',
        'Symbol': 'symbol'
    }, inplace=True)
    
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        
    expected_columns = [
        "Ticker", "close", "date", "high", "low", "month", "open", 
        "volume", "year", "daily_return", "cumulative_return", 
        "first_open", "last_close", "yearly_return", "volatility", 
        "monthly_return", "company", "sector", "symbol"
    ]   
    
    df = df[[col for col in expected_columns if col in df.columns]]
    
    df = df.where(pd.notnull(df), None)
        
    conn = pymysql.connect(
        host='localhost',
        user='root',
        password='Abcd1234',
        database='stock_analysis'
    )
    cursor = conn.cursor()

    # Table name
    table_name = "stock_data"

    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INT AUTO_INCREMENT PRIMARY KEY,
        Ticker VARCHAR(10),
        close FLOAT,
        date DATE,
        high FLOAT,
        low FLOAT,
        month VARCHAR(10),
        open FLOAT,                     
        volume BIGINT,
        year INT,
        daily_return FLOAT,
        cumulative_return FLOAT,
        first_open FLOAT,
        last_close FLOAT,
        yearly_return FLOAT,
        volatility FLOAT,
        monthly_return FLOAT,
        company VARCHAR(50),
        sector VARCHAR(50),
        symbol VARCHAR(20)
    );
    """
    cursor.execute(create_table_query)

    insert_query = f"""
    INSERT INTO {table_name} (
        Ticker, close, date, high, low, month, open, volume, year, daily_return,
        cumulative_return, first_open, last_close, yearly_return, volatility, monthly_return, company, sector, symbol
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
    """
    
    for _, row in df.iterrows():
       
        row_tuple = tuple(row)
        
        try:
            cursor.execute(insert_query, row_tuple)
        except Exception as e:
            print(f"Error inserting row: {row_tuple}\n{e}")
               
    conn.commit()
    cursor.close()
    conn.close()

    print(f"Data from {csv_file} stored in MySQL table '{table_name}'.")

csv_file = "D:/projects/stock/output/df.csv"
store_csv_in_mysql(csv_file)



Data from D:/projects/stock/output/df.csv stored in MySQL table 'stock_data'.
